In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [2]:
STORM_NUMBER = 2
TITLE = "19 October 2020"
SUBTITLE = "4 PM 19 October 2020"
sts = utc(2020, 10, 19, 17)
ets = utc(2020, 10, 20, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 15, 'z': 'yes', 'f': 'linear'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s, "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=0)
    return mp

In [24]:
cull = ['CHES2', 'FSDS2', 'CNNS2', 'WRNM5', 'ORCI4', 'SPRI4', 'WDOM5', 'NESN1', 'PTRI4', 'CKPI4', 'Z24']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
    ['Z23', 3],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=2)
res = mp.postprocess(filename='201019.png')
mp.close()

     state  wfo  val        lon        lat                             geo  \
58      IA  DMX  1.3 -94.110000  41.850000  POINT (487523.812 -332728.849)   
59      IA  DMX  1.2 -93.660000  41.540000  POINT (527260.400 -364339.181)   
60      IA  DVN  1.2 -90.540000  41.690000  POINT (783685.035 -322815.383)   
61      IA  DMX  1.1 -93.820000  41.880000  POINT (511228.697 -327636.476)   
62      IL  DVN  1.0 -89.980000  42.090000  POINT (824529.323 -273080.991)   
...    ...  ...  ...        ...        ...                             ...   
1590     Z  NaN  0.0 -94.731777  43.355474  POINT (425614.744 -169110.167)   
1591     Z  NaN  0.0 -93.621303  43.297744  POINT (515614.744 -169110.167)   
1592     Z  NaN  0.0 -91.406732  43.149415  POINT (695614.744 -169110.167)   
1593     Z  NaN  0.0 -90.303303  43.058896  POINT (785614.744 -169110.167)   
1594     Z  NaN  0.0 -94.696048  43.759140  POINT (425614.744 -124110.167)   

      used_for_analysis nwsli  plotme source  
58              